# Second Practical work

* Name Student 1: 
* NIA Student 1:
* Name Student 2:
* NIA Student 2:
  
This is second practical work skeleton.

It is divided in 3 parts_

1.   Python module load section: load all the needed modules
2.   Google Drive access and mounting
3.   Host load data
4.   GPU load





## Module load section
We will install the pycuda module in the colaboratory environment, to combine python and CUDA in our code. Because pycuda is not available by default, this task can take a few minutes.

As well we load numpy and matplotlib modules.

In [ ]:
!pip install pycuda

We import the Python modules which we will use in the notebook (numpy, pycuda and time module) 

In [ ]:
import  numpy  as  np
import  pycuda.autoinit
from    pycuda.compiler import SourceModule
import  pycuda.driver as  drv
import  pycuda.gpuarray as  gpuarray
from sys import getsizeof
import time

In [ ]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile

## Mount Google Drive and access grant
This part could be used to access to your Google Drive Data, **ONLY IF YOU ARE USING GOOGLE COLABORATORY**. Otherwise leave commented out. 

For security and privacy reasons, this permission is temporally, and will be requested every time you restart the notebook. 

Your Google Drive will be mounted in the "/content/drive" folder. The first Folder will be "My Drive".

You have to copy your images to a folder in the "My Drive" folder in your Google Drive.

To check your available files, use the cell indicated as "**List Files in Folder**"

In [ ]:
from os.path import join
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
# !ls /content/drive/My\ Drive/images

## IMAGES DEFINITION

The follow MYDRIVE variable will be used to address where are stored the images files.

Please, change it to your correct location.

All the images should be refered to this directory.

In [ ]:
MYDRIVE="./"

We define a variable IMAGE which is constructed joining the "MYDRIVE" varible plus the final path to your image.

In [ ]:
images = ["chess.jpg", "cloudsonjupi.jpg", "digits.jpg", "fence.jpg", "hand-x-ray.jpg"] 
IMAGE=join(MYDRIVE,images[4])

In [ ]:
print("Image to load: {0}".format(IMAGE))

Loads an image in a numpy variable, and show.

In [ ]:
image=np.array(Image.open(IMAGE))
plt.figure()
plt.imshow(image)

## FILTERS DEFINITION

Here we defines our sample filter masks.

In [ ]:
filter1=np.array([
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])
filter2=np.array([[0.5, 0 , -0.5]])
filter3=np.array([[0.5],[0],[-0.5]])

filter4=np.array([
    [1,2,1],
    [0,0,0],
    [-1,-2,-1]
])
filter5=np.array([
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]
])

## FILTER KERNEL

Here you have to define your image_filter GPU kernel.

In [ ]:
kernel  =  """
__global__ void image_filter( float * image,          //Source GPU array floating point 32 bits,
		                          float * filter_mask,    //Filter Mask GPU array 2D floating point 32 bits
                              float * filtered_image,  //Target GPU array 2D floating point 32 bits,
                              int NumRowsImg,         //Image Numrows,
                              int NumColsImg,         //Int32 Image Numcolumns,
                              int NumRowsFilter,       //Int32 Image NumRows filter mask,
                              int NumColsFilter       //Int32 Image NumCols filter mask
                              ) 
{
    // Initialization of variables --> information about the resulting filtered matrix
    int idxY; // It indicates the row of the element in the filtered matrix that the thread will modify
    int idxX; // It indicates the column of the element in the filtered matrix that the thread will modify
    int vX; // It indicates the element (in vector form, not in a matrix) in the filtered matrix that the thread will compute
    idxY = blockIdx.y*blockDim.y+threadIdx.y; //With this we calculate the row address in target matrix
    idxX = blockIdx.x*blockDim.x+threadIdx.x; // With this we calculate the column address in target matrix
        
    // We need to obtain information regarding which thread and block we are in
    // To obtain this information we do not need perform any calcualtion. The GPU internally knows this information

    int blockX = blockIdx.x;  // It indicates the block index that the thread corresponds with (x coordinate)
    int blockY = blockIdx.y; // It indicates the block index that the thread corresponds with (y coordinate)
    int threadIdxX = threadIdx.x; // It indicates the current thread index (x coordinate)
    int threadIdxY = threadIdx.y; // It indicates the current thread index (y coordinate)
    // In order not to access the global memory to access the filter values, we will create a shared variable (matrix), accessible
    // by all threads in the block. That matrix will be filled with the values of the global filter. This will accelerate the 
    // process of matrix multiplication, since the threads will not have to access to the global memory, but to the local shared one,
    // which is faster

    // We now compute the vector position in the filtered matrix corresponding with the idxY and idxX that indicated position in a matrix
    if ((idxX<NumRowsImg) && (idxY<NumColsImg)){ // check the thread is in a valid position (inside the range of the size of the image)
      vX=idxY*NumColsImg+idxX;
    }

    // Up to this point, we have --> initialized variables like block index, thread index,etc. And computed the element position that 
    // the thread is computing in the resulting filtered image.
    // We will now create a shared variable, which is accessible by all threads in the block. Its main advantage with respect to the 
    // global memory is that its access is faster. In this new local variable we will include the part of the image that the threads 
    // of this block will use to compute their corresponding filtered element. 

    __shared__ float tile_image[ %(block_s)s + %(rows_of_filter)s - 1][%(block_s)s + %(cols_of_filter)s - 1];
    // In tile_image matrix we will store the portion of the image needed by the threads. In order to know the part of the image needed:
    //    - We need at least the image corresponding with the block size. When computing the filtered result of a pixel in the middle area,
    //      the portion of the image (to multiply with the filter) needed will be in this region. However, those pixels closer to the border
    //      will need to access rows/columns not inside the block area. How to solve this? Making the tile_image bigger. We will need to consider
    //      as extra, the number of rows and columns of the filter. By doing this, pixels close to the border of the block will access the required
    //      pixels (of other blocks).
    // In order to understand better the tile_image, you need to image a matrix inside another matrix. The inner matrix is just the block dimensions.
    // However, the outer matrix also contains NumRowsFilter / 2 in the right and left sides and NumColsFilter / 2 in the upper and lower borders.
    // Now, the threads start filling the tile_image matrix by callling the image given as input. We will do this process in several steps. 

    // We create two new variables that indicate the borders
    int half_filter_x = NumColsFilter / 2;
    int half_filter_y = NumRowsFilter / 2;

    // FILLING INNER MATRIX
    // Each thread will fill the tile_image variable with its corresponding element in the image (Vx)
    tile_image[half_filter_y + threadIdxY][half_filter_x + threadIdxX] = image[vX];
    // Once each thread fills its element, the inner matrix will be filled.

    // Now, the threads corresponding with pixels close to the border of the block will be in charge of filling
    // the outter matrix of tile_image 
    // In order to understand better what we did, note that when we refer to "close to the border of the block"
    // we mean NumRowsFilter / 2 or less, for the right and left borders, and NumColsFilter / 2 or less, for the upper and lower borders

    // FILLING OUTTER MATRIX of tile_image. 
    // We will follow the next steps:
    //    1. Fill RIGHT border of tile_image
    //    2. Fill LEFT border of tile_image
    //    3. Fill UPPER border of tile_image
    //    4. Fill LOWER border of tile_image
    //    5. Corners of tile_image

    // 1. Fill RIGHT BORDER of tile_image
    // Note that only those threads that are close to the border of the block will write this part of the tile memory
    // In particular only those threads that are in the right border of the block
    if (threadIdxX > (blockDim.x - half_filter_x - 1)){  

      int symmetric_right = idxY * NumColsImg + (blockX + 1) * blockDim.x + (blockDim.x - threadIdxX - 1);
      // idxY * NumColsImg locates the start of the block (in vector shape)
      // (blockX - 1) * blockDim.x --> I go to the next block
      // (blockDim.x - threadIdxX - 1) --> I go to the end of the block and substract the thread index 
      // I get --> symmetric element (of the next block) with respect to the respective element assigned to the thread

      int rel_position = symmetric_right - idxY * NumColsImg; // Same as symmetric_right, but relative, not with respect other blocks
      int border = idxY * NumColsImg + NumColsImg - 1; // End of the block 

      // Fill the tile memory
      tile_image[half_filter_y + threadIdxY][(blockDim.x - threadIdxX) * 2 + threadIdxX + half_filter_x -1] = 
      (rel_position >= NumColsImg) ? image[border] : image[symmetric_right]; 
      // If the relative position is > Num columns, the thread is in a border of the image, and we fill the outter border with the same value. 
      // If not, we use the symmetric right value (provided by the next block).
    }
     

    // 2. Fill LEFT BORDER of tile_image
    // Note that only those threads that are close to the border of the block will write this part of the tile memory
    // In particular only those threads that are in the left border of the block
    if (threadIdxX < half_filter_x){  

      int symmetric_left = idxY * NumColsImg + (blockX - 1) * blockDim.x + (blockDim.x - threadIdxX - 1);
      // idxY * NumColsImg locates the start of the block (in vector shape)
      // (blockX - 1) * blockDim.x --> I go to the previous block
      // (blockDim.x - threadIdxX - 1) --> I go to the end of the block and substract the thread index 
      // I get --> symmetric element (of the previous block) with respect to the respective element assigned to the thread

      int rel_position = symmetric_left - idxY * NumColsImg; // Same as symmetric_left, but relative, not with respect other blocks
      int border = idxY * NumColsImg; // Start of the block 

      // Fill the tile memory
      tile_image[half_filter_y + threadIdxY][half_filter_x - threadIdxX - 1] = (rel_position < 0) ? image[border] : image[symmetric_left]; 
      // If the relative position is < 0, the thread is in a border, and we fill the outter border with the same value. If not, we use the symmetric value.
    }


    // 3. Fill UPPER BORDER of tile_image
    // Note that only those threads that are close to the border of the block will write this part of the tile memory
    // In particular only those threads that are in the upper border of the block
    if (threadIdxY < half_filter_y){ 

      int symmetric_up = ((blockY - 1) * blockDim.y + (blockDim.x - threadIdxY - 1)) * NumColsImg + idxX; // We go to the block that is 
      // located above this block (in the image), and obtain the symmetric element from that block with respect to the thread
      // Make use of (blockY - 1) * blockDim.y  to go to the previous block
      //  (blockDim.x - threadIdxY - 1)) * NumColsImg locates the start of the block
      // idxX --> locate the respective symmetric thread position within the upper block wrt its current position
      int rel_position = (blockY - 1) * blockDim.y + (blockDim.x - threadIdxY - 1); // Relative symmetric position
      int border = (blockY * blockDim.y) * NumColsImg + idxX; // Upper border of the image --> we just keep the value of the thread 

      // Fill the tile memory
      tile_image[half_filter_y - threadIdxY - 1][half_filter_x + threadIdxX] = 
      (rel_position < 0) ? image[border] : image[symmetric_up]; 
      // If the relative position is < 0, the thread is in a upper border of the image, and we fill the outter border with the same value. 
      // Otherwise, we use the symmetric upper value (provided by the upper block).
    }


    // 4. Fill LOWER BORDER of tile_image
    // Note that only those threads that are close to the lower border of the block will write this part of the tile memory
    // In particular only those threads that are in the lower border of the block
    if (threadIdxY > (blockDim.y - half_filter_y - 1)){  // check whether the thread is in the lower border

      int symmetric_down = ((blockY + 1) * blockDim.y + (blockDim.x - threadIdxY - 1)) * NumColsImg + idxX; // We go to the block that is 
      // located below this block (in the image), and obtain the symmetric element from that block with respect to the thread
      // Make use of (blockY + 1) * blockDim.y to go to the next block
      // (blockDim.x - threadIdxY - 1) * NumColsImg obtains the start of the lower block 
      // By adding idxX we obtain the symmetric element in the lower block wrt to the current one

      int rel_position = (blockY + 1) * blockDim.y + (blockDim.x - threadIdxY - 1); // Obtains relative position 
      int border = (blockY * blockDim.y + (blockDim.y - 1)) * NumColsImg + idxX; // Lower border of the image --> we just keep the value of the thread 
      // Difference wrt section 3 --> we add in the first term (blockDim.y - 1) --> in order to go to the next row

      // Fill the tile memory
      tile_image[(blockDim.y - threadIdxY) * 2 + threadIdxY + half_filter_y - 1][half_filter_x + threadIdxX] = 
      (rel_position >= NumColsImg) ? image[border] : image[symmetric_down]; 
      // If the relative position is larger than the number of columns, the thread is in a lower border of the image, and we fill the outter border with the same value. 
      // If not, we use the symmetric lower value (provided by the next block).
    }


    // 5. FILLING THE CORNERS
    // We will follow the next steps:
    //    1. Fill UPPER LEFT corner of tile_image
    //    2. Fill UPPER RIGHT corner of tile_image
    //    3. Fill LOWER LEFT corner of tile_image
    //    4. Fill LOWER RIGHT corner of tile_image

    // 5.1. Fill UPPER LEFT corner of tile_image

    if (threadIdxY < half_filter_y && threadIdxX < half_filter_x){ // Both x and y coordinates of the thread are located close to a corner
       // if the thread is in the upper left corner, it needs to satisfy the condition
       int corner_up_left = ((blockY - 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg + (blockX - 1)*blockDim.x + (blockDim.x-threadIdxX-1);
       // (blockY - 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg identifies the upper row
       // (blockX - 1)*blockDim.x + (blockDim.x-threadIdxX-1) identifies the first element of the upper row, that is, the symmetric wrt the x coordinate (of the previous block)
       int rel_y_position = (blockY - 1)*blockDim.y + (blockDim.x - threadIdxY - 1); // Obtains relative position of the previous block wrt y coordinate
       int rel_x_position = (blockX - 1)*blockDim.x + (blockDim.x - threadIdxX - 1); // Obtains relative position of the previous block wrt x coordinate

       int corner = (blockY * blockDim.y) * NumColsImg + blockX * blockDim.x; //
       tile_image[threadIdxY][threadIdxX] = (rel_y_position < 0 || rel_x_position < 0) ? image[corner] : image[corner_up_left];
       // Since the tile_image is bigger than the block size, by inserting the element in [threadIdxY][threadIdxX], we are pointing to the corner 
       // If any of the relative positions is negative, it means that we are in a corner of the image (not just a tile corner), and we will fill the element with 
       // the same value 
    }


    // 5.2. Fill UPPER RIGHT corner of tile_image
    if (threadIdxY < half_filter_y && threadIdxX > (blockDim.x - half_filter_x - 1)){ 
       // y coordinate of the thread in the upper part of the block, while x coordinate of the thread in the right part

       // if the thread is in the upper right corner, it needs to satisfy the condition
       int corner_up_right = ((blockY - 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg + (blockX + 1)*blockDim.x + (blockDim.x-threadIdxX-1);
       // (blockY - 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg identifies the previous row of the image
       // (blockX + 1)*blockDim.x + (blockDim.x-threadIdxX-1) identifies the next block and obtains the symmetric element wrt the x coordinate (of the next block)
       int rel_y_position = (blockY - 1)*blockDim.y + (blockDim.x - threadIdxY - 1); // Obtains relative position of the previous block wrt y coordinate
       int rel_x_position = (blockX + 1)*blockDim.x + (blockDim.x - threadIdxX - 1); // Obtains relative position of the next block wrt x coordinate

       int corner = (blockY * blockDim.y) * NumColsImg + NumColsImg - 1;
       tile_image[threadIdxY][(blockDim.x - 1) + half_filter_x + (blockDim.x - threadIdxX)] = (rel_y_position < 0 || rel_x_position >= NumColsImg) ? image[corner] : image[corner_up_right];
       // If we are located in the actual upper right corner of the image, we will fill the element with the corner value. However, if it is just a corner within the tile, with the corner_up_right value.
    }


    // 5.3. Fill LOWER LEFT corner of tile_image

    if (threadIdxY > (blockDim.y - half_filter_y - 1) && threadIdxX < half_filter_x){
       // if the thread is in the lower left corner, it needs to satisfy the condition
       int corner_lower_left = ((blockY + 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg + (blockX - 1)*blockDim.x + (blockDim.x-threadIdxX-1);
       // (blockY + 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg identifies the next row 
       // (blockX - 1)*blockDim.x + (blockDim.x-threadIdxX-1) obtains the symmetric value of the thread, of the previous block in the next row
       int rel_y_position = (blockY + 1)*blockDim.y + (blockDim.x - threadIdxY - 1); // relative position of the y coordinate of the thread within the next row
       int rel_x_position = (blockX - 1)*blockDim.x + (blockDim.x - threadIdxX-1); // relative position of the y coordinate of the thread within the previous block

       int corner = (blockY * blockDim.y + (blockDim.y - 1)) * NumColsImg + blockX * blockDim.x;
       tile_image[blockDim.y - 1][half_filter_x - threadIdxX - 1] = (rel_y_position >= NumColsImg || rel_x_position < 0) ? image[corner] : image[corner_lower_left];
    }


    // 5.4. Fill LOWER RIGHT corner of tile_image
    if (threadIdxY > (blockDim.y - half_filter_y - 1) && threadIdxX > (blockDim.x - half_filter_x - 1)){
       // if the thread is in the lower right corner, it needs to satisfy the condition
       int corner_down_right = ((blockY + 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg + (blockX + 1)*blockDim.x + (blockDim.x-threadIdxX-1);
       // (blockY + 1)*blockDim.y + (blockDim.x - threadIdxY - 1))*NumColsImg identifies the next row of the tile
       // (blockX + 1)*blockDim.x + (blockDim.x-threadIdxX-1) identifies the symmetric element wrt to the next block of the next row of the current thread
       int rel_y_position = (blockY + 1)*blockDim.y + (blockDim.x - threadIdxY - 1);
       int rel_x_position = (blockX + 1)*blockDim.x + (blockDim.x - threadIdxX - 1);

       int corner = (blockY * blockDim.y) * NumColsImg + NumColsImg - 1;
       tile_image[(blockDim.y - 1) + half_filter_y + (blockDim.y - threadIdxY)][(blockDim.x - 1) + half_filter_x + (blockDim.x - threadIdxX)] = (rel_y_position >= NumColsImg || rel_x_position >= NumColsImg) ? image[corner] : image[corner_down_right];
    }


    __shared__ float local_filter[%(rows_of_filter)s][%(cols_of_filter)s]; // Cration of the shared variable --> is a matrix of the same size as the filter
    // those threads whose index coincides with a valid position in the filter, will access the global memory and copy the respective filter
    // element in the shared filter
    if ((threadIdxX < NumColsFilter) && (threadIdxY < NumRowsFilter)){
      int elem_in_filter_mask =  threadIdxY * NumColsFilter + threadIdxX; // Given a row and a col in matrix form, compute the element in a 
      // vector (same reasoning as Vx)
      local_filter[threadIdxY][threadIdxX] = filter_mask[elem_in_filter_mask];
    }

    // We wait until all threads have filled the tiled memory with the corresponding image elements
    __syncthreads();

    // Now that we have the tile memory with the part of the image needed for the threads of the block, and the tile memory for the filter,
    // we can proceed to perform the filtering process

    float accumulator = 0;
    for (int row = 0; row < NumRowsFilter; row++){ // Iterate through the rows of the filter
      for (int col = 0; col < NumColsFilter; col++){ // Iterate through the columns of the filter
        accumulator += tile_image[row + threadIdx.y][col + threadIdx.x] * local_filter[row][col];
        // The accumulator stores the cummulative sum of an element of the tile with its respective filter element
      }
    }

    //Check whether the thread is in a valid position before modifying the filtered image
    if ((threadIdxY < NumRowsImg) && (threadIdxX < NumColsImg)){
      filtered_image[vX] = accumulator; 
      // If it is in a valid position, store the resulting filtered element in the filtered_image (vector) variable, in the previously computed vX position
    }
}
"""

And import the kernel definition to python

## SETUP EXPERIMENT

Recover the image shape, and select the two first items (rows and columns).

In [ ]:
image_shape=image.shape[0:2]

Set block size and grid size.

In [ ]:
block_s=16
block_size=(block_s,block_s,1)
grid_size=(int(np.ceil(image_shape[1]/block_s)),int(np.ceil(image_shape[0]/block_s)))

Allocates the target image memory.

In [ ]:
result_image=np.zeros(image.shape,dtype=np.uint8)

In [ ]:
depth = image.shape[2]

### Select the filter mask, and upload to the GPU device.

In [ ]:
filter_mask=filter1.astype(np.float32)
filter_mask_gpu=gpuarray.to_gpu(filter_mask)

In [ ]:
kernel_code = kernel_code_template % { 
    'rows_of_filter': filter_mask.shape[0],
    'cols_of_filter': filter_mask.shape[1],
    'block_s': block_s,
}
# compile the kernel code
kernel = SourceModule(kernel_code)

In [ ]:
image_filter = kernel.get_function('image_filter')

## Prepares the execution environment 

Allocate the temporal filtered memory layer, where will be store the filtered image.

In [ ]:
filtered_image_layer_gpu = gpuarray.zeros(image_shape,np.float32)

Here defines the main block.
Walk through the layers. Copying the selected layer to temporal image_layer, upload the layer to the GPU and invokes the image_filter kernel.

In [ ]:
%%timeit -r 10 -n 10 -o
for layer in range(depth):
  image_layer = np.zeros(image_shape,dtype=np.float32) #Allocate the temporal image layer
  image_layer=(image[:,:,layer]).astype(np.float32)    #Copy from the orignal image the selected layer into the temporal memory
  image_layer_gpu=gpuarray.to_gpu(image_layer)         #Uploads the image layer to the GPU
  filtered_image_layer_gpu.fill(0.0)                   #Fills with 0 the target memory in the GPU
  image_filter(                                        #invokes the kernel
          image_layer_gpu,
          filter_mask_gpu,
          filtered_image_layer_gpu,
          np.int32(image_shape[0]),
          np.int32(image_shape[1]),
          np.int32(filter_mask.shape[0]),
          np.int32(filter_mask.shape[1]),
          block=block_size,
          grid=grid_size
        )
  filtered_image_layer=filtered_image_layer_gpu.get() #Download from the GPU the filtered image
  result_image[:,:,layer] = filtered_image_layer.astype(np.uint8) #Copy the filtered image to the final image store, 
                                                                  #casting the data type from double to uint8

*NOTE* The follow code fetch from the stack the exectution statistics produced by the magic metacommand %%timeit and stores it in a python variable.

In [ ]:
statistics=_

# Show results


In [ ]:
plt.figure()
plt.imshow(result_image)

In [ ]:
print("Best execution time: {}".format(statistics.best))
print("Worst execution time: {}".format(statistics.worst))
print("Mean execution time: {mean} ({stdev} std dev)".format(mean=statistics.average,stdev=statistics.stdev))

# CONCLUSIONS